<a href="https://colab.research.google.com/github/Srinathi117/Pancreatic-Cancer_app/blob/main/Pancreatic_Checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import zipfile
import os

# Upload zip file
uploaded = files.upload()

# Extract the uploaded zip file
for file_name in uploaded.keys():
    print(f"Extracting: {file_name}")
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')  # Extract to folder

# Check the extracted files
os.listdir('/content/dataset')


Saving archive (5).zip to archive (5) (1).zip
Extracting: archive (5) (1).zip


['DATASET']

In [ ]:
# ================== PREPROCESSING PIPELINE FOR IMAGE DATASET ==================
# Uses: TensorFlow image_dataset_from_directory + normalization + augmentation
# Dataset location: /content/dataset/DATASET  (already extracted)

import os
import tensorflow as tf
from tensorflow.keras import layers

# ---------- 1. BASIC SETTINGS ----------
base_dir   = "/content/dataset/DATASET"  # change only if your folder name is different
img_height = 224
img_width  = 224
batch_size = 32
val_split  = 0.2
seed       = 42

print("Base dataset folder:", base_dir)
print("Folders inside (classes should be here):", os.listdir(base_dir))

# ---------- 2. CREATE TRAIN & VALIDATION DATASETS ----------
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=val_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=val_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds.class_names
print("\nDetected Classes:", class_names)

# ---------- 3. DEFINE NORMALIZATION & AUGMENTATION ----------
AUTOTUNE = tf.data.AUTOTUNE

# Scale pixels from [0,255] -> [0,1]
normalization_layer = layers.Rescaling(1./255)

# Data augmentation (applied only on training set)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),   # flip images
    layers.RandomRotation(0.1),        # rotate a bit
    layers.RandomZoom(0.1),            # zoom a bit
])

def preprocess_train(images, labels):
    images = data_augmentation(images)
    images = normalization_layer(images)
    return images, labels

def preprocess_val(images, labels):
    images = normalization_layer(images)
    return images, labels

# ---------- 4. APPLY PREPROCESSING + PERFORMANCE OPTIMIZATION ----------
train_ds = train_ds.map(preprocess_train, num_parallel_calls=AUTOTUNE)
val_ds   = val_ds.map(preprocess_val,   num_parallel_calls=AUTOTUNE)

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("\n✅ Preprocessing complete. 'train_ds' and 'val_ds' are ready for model training!")

# ---------- 5. (OPTIONAL) QUICK SHAPE CHECK ----------
for images, labels in train_ds.take(1):
    print("Sample batch image shape:", images.shape)
    print("Sample batch label shape:", labels.shape)
    break

# Now you can use:
# model.fit(train_ds, validation_data=val_ds, epochs=10)


Base dataset folder: /content/dataset/DATASET
Folders inside (classes should be here): ['test', 'train']
Found 1411 files belonging to 2 classes.
Using 1129 files for training.
Found 1411 files belonging to 2 classes.
Using 282 files for validation.

Detected Classes: ['test', 'train']

✅ Preprocessing complete. 'train_ds' and 'val_ds' are ready for model training!
Sample batch image shape: (32, 224, 224, 3)
Sample batch label shape: (32,)


In [ ]:
# ================== MODEL, TRAINING & TESTING ==================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# ---------- 1. GET NUMBER OF CLASSES ----------
num_classes = len(class_names)
print("Number of classes:", num_classes)
print("Classes:", class_names)

# ---------- 2. BUILD TRANSFER LEARNING MODEL (MobileNetV2) ----------
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze base model (only train top layers first)
base_model.trainable = False

inputs = keras.Input(shape=(img_height, img_width, 3))

# If your dataset is already normalized in pipeline, you can skip this.
# But to be safe, we add a scaling layer compatible with MobileNetV2.
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)  # prevent overfitting
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)

# ---------- 3. COMPILE MODEL ----------
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ---------- 4. TRAIN THE MODEL ----------
epochs = 10  # you can increase this

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

# ---------- 5. EVALUATE (TEST) ON VALIDATION SET ----------
print("\nEvaluating on validation (test) dataset:")
test_loss, test_acc = model.evaluate(val_ds)
print(f"Validation/Test Loss: {test_loss:.4f}")
print(f"Validation/Test Accuracy: {test_acc:.4f}")

# ---------- 6. OPTIONAL: FINE-TUNE SOME LAYERS ----------
# Unfreeze some top layers of the base model for fine-tuning
fine_tune = False  # change to True if you want fine-tuning

if fine_tune:
    base_model.trainable = True

    # Freeze bottom N layers, train top layers
    fine_tune_at = 100  # adjust depending on model depth
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # smaller LR
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    print("\nStarting fine-tuning...")
    history_fine = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=5
    )

# ---------- 7. PREDICT ON A SINGLE IMAGE ----------
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_single_image(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # shape: (1, h, w, 3)

    # Preprocess same as training
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class = class_names[predicted_class_index]
    confidence = predictions[0][predicted_class_index]

    print(f"Image: {img_path}")
    print(f"Predicted class: {predicted_class} (confidence: {confidence:.4f})")

# Example usage (change path):
# predict_single_image("/content/dataset/DATASET/Normal/image1.png")


Number of classes: 2
Classes: ['test', 'train']
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         2,562 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.6748 - loss: 0.6682 - val_accuracy: 0.7234 - val_loss: 0.6032
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.6592 - loss: 0.6771 - val_accuracy: 0.7234 - val_loss: 0.5986
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.6427 - loss: 0.7021 - val_accuracy: 0.7234 - val_loss: 0.5985
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.6638 - loss: 0.6473 - val_accuracy: 0.7234 - val_loss: 0.5971
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.6660 - loss: 0.6695 - val_accuracy: 0.7234 - val_loss: 0.5963
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 38s 978ms/step - accuracy: 0.6700 - loss: 0.6733 - val_accuracy: 0.7234 - val_loss: 0.5942
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.6881 - loss: 0.6659 - val_accuracy: 0.7234 - val_loss: 0.5946
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 34s 957ms/step - accuracy: 0.6691 - loss: 0.6689 - val_accuracy: 0.7234 - val

In [ ]:
# ============================
# 1. INSTALL REQUIRED LIBRARIES
# ============================
# Run this cell once
!pip install -q torch torchvision torchaudio
!pip install -q torch-geometric
!pip install -q scikit-learn

# ============================
# 2. IMPORTS & BASIC SETUP
# ============================
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Path to your extracted dataset
data_dir = "/content/dataset/DATASET"   # make sure this folder contains class subfolders
print("Classes (folders) in dataset:", os.listdir(data_dir))

# ============================
# 3. IMAGE DATASET & FEATURE EXTRACTION (CNN BACKBONE)
# ============================

# Transform: resize to 224x224 and normalize (ImageNet style)
img_size = 224
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet means
        std=[0.229, 0.224, 0.225]    # ImageNet stds
    ),
])

# ImageFolder expects structure: DATASET/class_name/..images..
img_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
num_classes = len(img_dataset.classes)
print("Detected classes:", img_dataset.classes)
print("Number of images:", len(img_dataset))

# Dataloader just for feature extraction
img_loader = DataLoader(img_dataset, batch_size=64, shuffle=False, num_workers=2)

# Pretrained ResNet18 backbone – we use it as feature extractor
backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
backbone.fc = nn.Identity()  # remove classification head -> get feature vector
backbone = backbone.to(device)
backbone.eval()

# Extract features for ALL images
all_features = []
all_labels = []

with torch.no_grad():
    for imgs, labels in img_loader:
        imgs = imgs.to(device)
        feats = backbone(imgs)           # [batch, feat_dim]
        all_features.append(feats.cpu())
        all_labels.append(labels)

X = torch.cat(all_features, dim=0)      # [N, feat_dim]
y = torch.cat(all_labels, dim=0)        # [N]
print("Feature matrix shape:", X.shape) # e.g., [N, 512] for ResNet18

# ============================
# 4. BUILD A GRAPH (NODES = IMAGES, EDGES = SIMILAR IMAGES)
# ============================

# Build k-NN graph using feature vectors
k = 8  # number of neighbors per node
features_np = X.numpy()

nbrs = NearestNeighbors(n_neighbors=k + 1, metric='euclidean').fit(features_np)
distances, indices = nbrs.kneighbors(features_np)  # indices: [N, k+1]

edge_index_list = []
N = indices.shape[0]

for i in range(N):
    for j in indices[i, 1:]:  # skip the first one (it is the node itself)
        edge_index_list.append([i, j])
        edge_index_list.append([j, i])  # make it undirected

edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()
print("edge_index shape:", edge_index.shape)  # [2, num_edges]

# ============================
# 5. TRAIN/VAL/TEST SPLIT & MASKS
# ============================

idx = np.arange(N)

# 20% test
idx_train_val, idx_test = train_test_split(
    idx, test_size=0.2, stratify=y.numpy(), random_state=42
)

# From remaining, 20% of that as val (so final ~64% train, 16% val, 20% test)
idx_train, idx_val = train_test_split(
    idx_train_val, test_size=0.2, stratify=y.numpy()[idx_train_val], random_state=42
)

train_mask = torch.zeros(N, dtype=torch.bool)
val_mask   = torch.zeros(N, dtype=torch.bool)
test_mask  = torch.zeros(N, dtype=torch.bool)

train_mask[idx_train] = True
val_mask[idx_val]     = True
test_mask[idx_test]   = True

print(f"Train nodes: {train_mask.sum().item()}, "
      f"Val nodes: {val_mask.sum().item()}, "
      f"Test nodes: {test_mask.sum().item()}")

# ============================
# 6. CREATE PYTORCH GEOMETRIC DATA OBJECT
# ============================
graph_data = Data(
    x=X,                 # node features [N, feat_dim]
    edge_index=edge_index,
    y=y,                 # labels [N]
    train_mask=train_mask,
    val_mask=val_mask,
    test_mask=test_mask
).to(device)

# ============================
# 7. DEFINE GNN MODEL (2-LAYER GCN)
# ============================

class PancreaticGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)
        self.dropout = dropout

    def forward(self, x, edge_index):
        # First GCN layer + ReLU
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = nn.functional.dropout(x, p=self.dropout, training=self.training)
        # Second GCN layer
        x = self.conv2(x, edge_index)
        return x

in_channels = graph_data.x.size(1)
hidden_channels = 128

model = PancreaticGCN(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    num_classes=num_classes,
    dropout=0.5
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

print(model)

# ============================
# 8. TRAINING & EVALUATION LOOPS
# ============================

def train():
    model.train()
    optimizer.zero_grad()
    out = model(graph_data.x, graph_data.edge_index)
    loss = criterion(out[graph_data.train_mask], graph_data.y[graph_data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def evaluate(mask):
    model.eval()
    out = model(graph_data.x, graph_data.edge_index)
    pred = out.argmax(dim=1)
    correct = (pred[mask] == graph_data.y[mask]).sum().item()
    total = int(mask.sum())
    acc = correct / total if total > 0 else 0
    return acc

epochs = 50  # you can increase this
for epoch in range(1, epochs + 1):
    loss = train()
    train_acc = evaluate(graph_data.train_mask)
    val_acc = evaluate(graph_data.val_mask)
    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | "
              f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

# Final test accuracy
test_acc = evaluate(graph_data.test_mask)
print(f"\n✅ Final Test Accuracy: {test_acc:.4f}")

# ============================
# 9. PREDICT ON A NEW IMAGE
# ============================

from PIL import Image

def predict_pancreatic_image(img_path):
    """
    1) Load image
    2) Extract CNN feature using the same backbone
    3) Use GNN model to predict class based on graph neighborhood
       (we approximate by plugging into model without adding new edges).
    """
    model.eval()
    backbone.eval()

    # Load and preprocess image
    img = Image.open(img_path).convert("RGB")
    img_t = transform(img).unsqueeze(0).to(device)  # [1, 3, H, W]

    with torch.no_grad():
        feat = backbone(img_t)  # [1, feat_dim]

        # For a new image, we don't have edges; simplest is to classify using MLP-like behavior.
        # We approximate by passing it through GNN with NO neighbors (edge_index doesn't matter).
        # For proper behavior you would add this node to graph and create edges via kNN again.
        # Here we just reuse the learned weight structure in a simple way:

        # Fake edge_index: self-loop only
        new_edge_index = torch.tensor([[0], [0]], dtype=torch.long).to(device)

        out = model(feat, new_edge_index)      # [1, num_classes]
        probs = torch.softmax(out, dim=1)[0]
        pred_class_idx = int(torch.argmax(probs))
        pred_class_name = img_dataset.classes[pred_class_idx]
        confidence = float(probs[pred_class_idx])

    print(f"Image: {img_path}")
    print(f"Predicted class: {pred_class_name} "
          f"(confidence: {confidence:.4f})")
    return pred_class_name, confidence

# Example usage (change image path):
# predict_pancreatic_image("/content/dataset/DATASET/Normal/sample1.png")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.6 MB/s eta 0:00:00
Using device: cpu
Classes (folders) in dataset: ['test', 'train']
Detected classes: ['test', 'train']
Number of images: 1411
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 142MB/s]


Feature matrix shape: torch.Size([1411, 512])
edge_index shape: torch.Size([2, 22576])
Train nodes: 902, Val nodes: 226, Test nodes: 283
PancreaticGCN(
  (conv1): GCNConv(512, 128)
  (conv2): GCNConv(128, 2)
)
Epoch 001 | Loss: 0.8105 | Train Acc: 0.7084 | Val Acc: 0.7080
Epoch 005 | Loss: 0.5901 | Train Acc: 0.7827 | Val Acc: 0.7788
Epoch 010 | Loss: 0.3975 | Train Acc: 0.8625 | Val Acc: 0.8717
Epoch 015 | Loss: 0.3270 | Train Acc: 0.8625 | Val Acc: 0.8717
Epoch 020 | Loss: 0.3085 | Train Acc: 0.8470 | Val Acc: 0.8142
Epoch 025 | Loss: 0.2977 | Train Acc: 0.8625 | Val Acc: 0.8717
Epoch 030 | Loss: 0.2974 | Train Acc: 0.8559 | Val Acc: 0.8363
Epoch 035 | Loss: 0.2874 | Train Acc: 0.8625 | Val Acc: 0.8717
Epoch 040 | Loss: 0.2782 | Train Acc: 0.8625 | Val Acc: 0.8673
Epoch 045 | Loss: 0.2759 | Train Acc: 0.8614 | Val Acc: 0.8496
Epoch 050 | Loss: 0.2812 | Train Acc: 0.8636 | Val Acc: 0.8717

✅ Final Test Accuracy: 0.8410


In [ ]:
# ================= PREDICT CANCER OR NOT (UPLOAD IN COLAB) ==================
from google.colab import files
from PIL import Image
import torch

# NOTE: This requires that:
# - model, backbone, transform, img_dataset, and device are already defined
#   (from the GNN training code)

def predict_pancreatic_image(img_path):
    model.eval()
    backbone.eval()

    img = Image.open(img_path).convert("RGB")
    img_t = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        feat = backbone(img_t)

        # Fake edge (self-loop) for single inference
        edge_index = torch.tensor([[0], [0]], dtype=torch.long).to(device)

        out = model(feat, edge_index)
        probs = torch.softmax(out, dim=1)[0]
        pred_idx = int(torch.argmax(probs))
        pred_class = img_dataset.classes[pred_idx]
        confidence = float(probs[pred_idx])

    return pred_class, confidence


def cancer_or_not(pred_class):
    # Customize to your folder names
    # If class name contains word "cancer" → positive
    if "cancer" in pred_class.lower():
        return "⚠️ The model predicts: CANCER"
    else:
        return "✅ The model predicts: NO CANCER"


# ========== UPLOAD + RUN PREDICTION ==========
uploaded = files.upload()

for filename in uploaded.keys():
    img_path = '/content/' + filename

    pred_class, conf = predict_pancreatic_image(img_path)
    result = cancer_or_not(pred_class)

    print("\n===============================")
    print(f"Image: {filename}")
    print(f"Predicted Class: {pred_class}")
    print(f"Confidence: {conf:.4f}")
    print(result)
    print("===============================")


Saving 1-004.jpg to 1-004.jpg

Image: 1-004.jpg
Predicted Class: train
Confidence: 0.6336
✅ The model predicts: NO CANCER


In [ ]:
# ==== SANITY CHECK: WHICH CLASS IS WHICH? ====
print("Class index mapping:")
for idx, name in enumerate(img_dataset.classes):
    print(idx, "->", name)

# ==== SIMPLE EVALUATION ON TEST NODES ====
import torch

model.eval()
with torch.no_grad():
    out = model(graph_data.x, graph_data.edge_index)
    pred = out.argmax(dim=1)

# Confusion-like counts for each class in test set
from collections import Counter

true_labels = graph_data.y[graph_data.test_mask].cpu().numpy()
pred_labels = pred[graph_data.test_mask].cpu().numpy()

print("\nTest set size:", len(true_labels))

print("\nTrue label counts (test):", Counter(true_labels))
print("Pred label counts (test):", Counter(pred_labels))

# How many are correct?
correct = (true_labels == pred_labels).sum()
print("\nCorrect predictions on test:", correct)
print("Test accuracy (again):", correct / len(true_labels))


Class index mapping:
0 -> test
1 -> train

Test set size: 283

True label counts (test): Counter({np.int64(1): 200, np.int64(0): 83})
Pred label counts (test): Counter({np.int64(1): 235, np.int64(0): 48})

Correct predictions on test: 238
Test accuracy (again): 0.8409893992932862


In [ ]:
# ==============================================
# PANCREATIC CANCER GNN: TRAIN + TEST (FIXED PATHS)
# ==============================================

# ---------- 0. INSTALL LIBRARIES (RUN ONCE) ----------
!pip install -q torch torchvision torchaudio
!pip install -q torch-geometric
!pip install -q scikit-learn

# ---------- 1. IMPORTS & BASIC SETUP ----------
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import datasets, transforms, models

from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np
from collections import Counter

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------- 2. PATHS ----------
data_root = "/content/dataset/DATASET"   # your extracted folder
print("Inside DATASET:", os.listdir(data_root))

train_root_base = os.path.join(data_root, "train")
test_root_base  = os.path.join(data_root, "test")

print("Raw train_root_base:", train_root_base)
print("Raw test_root_base:", test_root_base)

def find_class_root(root):
    """If there is only one subfolder (like 'train' or 'test'), go one level deeper."""
    subs = [d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]
    print(f"\nChecking root: {root}")
    print(" Subfolders:", subs)
    if len(subs) == 1:
        new_root = os.path.join(root, subs[0])
        print(" -> Only one subfolder, using inner folder as class root:", new_root)
        return new_root
    else:
        print(" -> Using this as class root")
        return root

train_root = find_class_root(train_root_base)
test_root  = find_class_root(test_root_base)

print("\nFinal train_root:", train_root)
print("Final test_root:", test_root)

print("\nTrain classes folders:", os.listdir(train_root))
print("Test classes folders:", os.listdir(test_root))

# ---------- 3. TRANSFORMS ----------
img_size = 224
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet stats
        std=[0.229, 0.224, 0.225]
    ),
])

# ---------- 4. DATASETS ----------
train_dataset = datasets.ImageFolder(root=train_root, transform=transform)
test_dataset  = datasets.ImageFolder(root=test_root,  transform=transform)

print("\nTrain classes:", train_dataset.classes)
print("Test classes:", test_dataset.classes)

# Ensure same class mapping
assert train_dataset.classes == test_dataset.classes, "Train/Test classes mismatch!"

class_names = train_dataset.classes
num_classes = len(class_names)

print("\nClass index mapping:")
for idx, name in enumerate(class_names):
    print(idx, "->", name)
print("Number of classes:", num_classes)

# Merge train + test into one dataset for building a single graph
full_dataset = ConcatDataset([train_dataset, test_dataset])
print("\nTotal images (train + test):", len(full_dataset))

# ---------- 5. FEATURE EXTRACTION (RESNET18 BACKBONE) ----------
loader = DataLoader(full_dataset, batch_size=64, shuffle=False, num_workers=2)

backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
backbone.fc = nn.Identity()   # remove final classifier -> feature extractor
backbone = backbone.to(device)
backbone.eval()

all_features = []
all_labels = []

with torch.no_grad():
    for imgs, labels in loader:
        imgs = imgs.to(device)
        feats = backbone(imgs)    # [B, feat_dim]
        all_features.append(feats.cpu())
        all_labels.append(labels)

X = torch.cat(all_features, dim=0)   # [N, feat_dim]
y = torch.cat(all_labels, dim=0)     # [N]

print("\nFeature matrix shape:", X.shape)
print("Labels shape:", y.shape)

# ---------- 6. BUILD k-NN GRAPH ----------
k = 8   # neighbors per node
features_np = X.numpy()

nbrs = NearestNeighbors(n_neighbors=k + 1, metric='euclidean').fit(features_np)
distances, indices = nbrs.kneighbors(features_np)

edge_index_list = []
N = indices.shape[0]

for i in range(N):
    for j in indices[i, 1:]:   # skip itself
        edge_index_list.append([i, j])
        edge_index_list.append([j, i])   # undirected

edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()
print("edge_index shape:", edge_index.shape)  # [2, num_edges]

# ---------- 7. TRAIN/VAL/TEST SPLIT ----------
idx = np.arange(N)

# 20% test
idx_train_val, idx_test = train_test_split(
    idx, test_size=0.2, stratify=y.numpy(), random_state=42
)

# From remaining: 20% as val => 64% train, 16% val, 20% test
idx_train, idx_val = train_test_split(
    idx_train_val, test_size=0.2, stratify=y.numpy()[idx_train_val], random_state=42
)

train_mask = torch.zeros(N, dtype=torch.bool)
val_mask   = torch.zeros(N, dtype=torch.bool)
test_mask  = torch.zeros(N, dtype=torch.bool)

train_mask[idx_train] = True
val_mask[idx_val]     = True
test_mask[idx_test]   = True

print(f"\nTrain nodes: {train_mask.sum().item()}, "
      f"Val nodes: {val_mask.sum().item()}, "
      f"Test nodes: {test_mask.sum().item()}")

# ---------- 8. GRAPH DATA OBJECT ----------
graph_data = Data(
    x=X,
    edge_index=edge_index,
    y=y,
    train_mask=train_mask,
    val_mask=val_mask,
    test_mask=test_mask
).to(device)

# ---------- 9. DEFINE GNN MODEL ----------
class PancreaticGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = nn.functional.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x

in_channels = graph_data.x.size(1)
hidden_channels = 128

model = PancreaticGCN(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    num_classes=num_classes,
    dropout=0.5
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

print("\nGNN model:\n", model)

# ---------- 10. TRAIN & EVALUATE ----------
def train_step():
    model.train()
    optimizer.zero_grad()
    out = model(graph_data.x, graph_data.edge_index)
    loss = criterion(out[graph_data.train_mask], graph_data.y[graph_data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def accuracy(mask):
    model.eval()
    out = model(graph_data.x, graph_data.edge_index)
    pred = out.argmax(dim=1)
    correct = (pred[mask] == graph_data.y[mask]).sum().item()
    total = int(mask.sum())
    return correct / total if total > 0 else 0.0

epochs = 50   # increase if needed

for epoch in range(1, epochs + 1):
    loss = train_step()
    train_acc = accuracy(graph_data.train_mask)
    val_acc   = accuracy(graph_data.val_mask)
    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | "
              f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

test_acc = accuracy(graph_data.test_mask)
print(f"\n✅ Final Test Accuracy: {test_acc:.4f}")

# Label distribution on test
model.eval()
with torch.no_grad():
    out = model(graph_data.x, graph_data.edge_index)
    pred = out.argmax(dim=1)

true_labels = graph_data.y[graph_data.test_mask].cpu().numpy()
pred_labels = pred[graph_data.test_mask].cpu().numpy()

print("\nTrue label counts (test):", Counter(true_labels))
print("Pred label counts (test):", Counter(pred_labels))

# ---------- 11. OPTIONAL: SINGLE IMAGE PREDICTION ----------
def predict_single_image(img_path):
    """
    Predict class for a new image using the trained GNN.
    (Approximate by using a self-loop graph for the new node.)
    """
    model.eval()
    backbone.eval()

    img = Image.open(img_path).convert("RGB")
    img_t = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        feat = backbone(img_t)  # [1, feat_dim]
        edge_index_new = torch.tensor([[0], [0]], dtype=torch.long).to(device)
        out = model(feat, edge_index_new)       # [1, num_classes]
        probs = torch.softmax(out, dim=1)[0]
        pred_idx = int(torch.argmax(probs))
        pred_class = class_names[pred_idx]
        conf = float(probs[pred_idx])

    print(f"\nImage: {img_path}")
    print(f"Predicted class: {pred_class} (index {pred_idx})")
    print(f"Confidence: {conf:.4f}")
    return pred_class, conf

# Example usage after training:
# predict_single_image("/content/dataset/DATASET/test/test/<CLASS>/<IMAGE>.png")


Using device: cpu
Inside DATASET: ['test', 'train']
Raw train_root_base: /content/dataset/DATASET/train
Raw test_root_base: /content/dataset/DATASET/test

Checking root: /content/dataset/DATASET/train
 Subfolders: ['train']
 -> Only one subfolder, using inner folder as class root: /content/dataset/DATASET/train/train

Checking root: /content/dataset/DATASET/test
 Subfolders: ['test']
 -> Only one subfolder, using inner folder as class root: /content/dataset/DATASET/test/test

Final train_root: /content/dataset/DATASET/train/train
Final test_root: /content/dataset/DATASET/test/test

Train classes folders: ['normal', 'pancreatic_tumor']
Test classes folders: ['normal', 'pancreatic_tumor']

Train classes: ['normal', 'pancreatic_tumor']
Test classes: ['normal', 'pancreatic_tumor']

Class index mapping:
0 -> normal
1 -> pancreatic_tumor
Number of classes: 2

Total images (train + test): 1411

Feature matrix shape: torch.Size([1411, 512])
Labels shape: torch.Size([1411])
edge_index shape: to

In [ ]:
# ================= PREDICT: CANCER OR NO CANCER (UPLOAD IMAGE IN COLAB) =================
from google.colab import files
from PIL import Image
import torch

# 🔴 IMPORTANT: based on your mapping
# 0 -> normal
# 1 -> pancreatic_tumor  (this is cancer)
CANCER_CLASS_INDICES = [1]

print("Class names:", class_names)
print("Cancer class indices:", CANCER_CLASS_INDICES)

def predict_pancreatic_image(img_path):
    """
    Use the trained backbone + GNN model to predict class for one image.
    """
    model.eval()
    backbone.eval()

    img = Image.open(img_path).convert("RGB")
    img_t = transform(img).unsqueeze(0).to(device)   # [1, 3, H, W]

    with torch.no_grad():
        # CNN feature
        feat = backbone(img_t)                      # [1, feat_dim]

        # Self-loop graph for single node
        edge_index_new = torch.tensor([[0], [0]], dtype=torch.long).to(device)

        out = model(feat, edge_index_new)           # [1, num_classes]
        probs = torch.softmax(out, dim=1)[0]        # [num_classes]
        pred_idx = int(torch.argmax(probs))
        pred_class = class_names[pred_idx]
        conf = float(probs[pred_idx])

    return pred_idx, pred_class, conf

def cancer_or_not(pred_idx):
    if pred_idx in CANCER_CLASS_INDICES:
        return "⚠️ The model predicts: CANCER (pancreatic tumor)."
    else:
        return "✅ The model predicts: NO CANCER (normal)."

# ------------ UPLOAD + PREDICT ------------
uploaded = files.upload()   # choose image file(s)

for filename in uploaded.keys():
    img_path = "/content/" + filename

    pred_idx, pred_class, conf = predict_pancreatic_image(img_path)
    result_text = cancer_or_not(pred_idx)

    print("\n===================================")
    print(f"Image: {filename}")
    print(f"Predicted class label: {pred_class} (index {pred_idx})")
    print(f"Confidence: {conf:.4f}")
    print(result_text)
    print("===================================")


Class names: ['normal', 'pancreatic_tumor']
Cancer class indices: [1]


Saving 1-002.jpg to 1-002.jpg

Image: 1-002.jpg
Predicted class label: normal (index 0)
Confidence: 0.9970
✅ The model predicts: NO CANCER (normal).
